In [1]:
import os
import re
import json
from tqdm.notebook import tqdm
from textwrap import dedent

import numpy as np
import pandas as pd
from scipy import ndimage
from scipy import optimize 
from sklearn.preprocessing import MinMaxScaler, PowerTransformer

import matplotlib.pyplot as plt

import utils

def scatter(w, k):
    return k * (1 / w**4)

def r_squared(y, y_hat):
    residuals = y - y_hat
    ss_res = np.sum(residuals**2)
    ss_tot = np.sum((y-np.mean(y))**2)
    r_squared = 1 - (ss_res / ss_tot)
    return r_squared


root_dir_walk = os.walk('..')

plate_files = []

experiment_configs = {}

for dirpath, dirnames, filenames in root_dir_walk:
    if (e := re.search('\.\./([0-9]+)', dirpath)) is not None:
        experiment_number = int(e.group(1))
        data = {}
        if 1 <= experiment_number <= 22:
            if experiment_number not in experiment_configs.keys():
                for filename in filenames:
                    fpath = os.path.join(dirpath, filename)
                    if filename.split('.')[-1] == 'CSV' and 'pathlength' not in filename.lower():
                        plate_files.append(fpath)
                    if re.search('experiment-[0-9]+-summary\.csv', filename) is not None:
                        data['summary_fname'] = filename 
                        data['summary_path'] = fpath
                    if re.search('config\_[0-9]+\.json', filename) is not None:
                        data['config_fname'] = filename 
                        with open(fpath, 'r') as f:
                            data['config'] = json.load(f)
                if data != {}:
                    experiment_configs[experiment_number] = data

configs_df = pd.concat([pd.read_csv(experiment_configs[i]['summary_path']) for i in experiment_configs]).reset_index(drop=True)

In [2]:
df = pd.concat([utils.bmg.parse_bmg(i) for i in plate_files])

df_corr = df.subtract(df[800], axis=0)

df_clear = df_corr.loc[df_corr[280] < 1, :]
df_normal = df_corr.loc[df_corr[280] > 1, :]

In [3]:
sorted([i for i in experiment_configs.keys() if 'experiments' not in experiment_configs[i]['config'].keys()])

[]

In [4]:
experiment_configs[4]['config']['experiments']

{'plate_1': {'file': 'data/20190618_Assay2.csv',
  'plate_type': 'Thermo Polystyrene (262160)',
  'columns': {'13': 'DMSO',
   '14': 'Lauric acid',
   '15': 'Lauric acid',
   '16': 'Arachadionic acid',
   '17': 'Arachadionic acid',
   '18': '4-Phenylimidazole',
   '19': '4-Phenylimidazole'}}}

## Trim $A < 300$

In [5]:
df_normal = df_normal.loc[:, 300:]



## Zero Traces at 800 nm

Commonly, traces are transformed in the $A$ direction by an amount equal to $A_{800}$

This is because of?

Join configs to traces

Uniformly subtracting $A_{800}$ from each trace cancels this effect, which has precedent?

## Distribution:

In [6]:
plt.hist(df[800], bins=100)
plt.xlim(0, 0.3)
plt.xlabel('$A_{800}$')
plt.ylabel('Count')
plt.title('Distribution of $A_{800}$ Over All Development Experiments')
plt.show()

## Figure: Before and After $A_{800}$ Normalization

In [7]:
df_normal_smooth = pd.DataFrame(ndimage.gaussian_filter1d(df_normal, 
                                                          sigma=5,
                                                          axis=1,), 
                                columns=df_normal.columns, 
                                index=df_normal.index,
                               )

df_normal_smooth =  df_normal_smooth.subtract(df_normal_smooth[800], axis=0)

# auc_smooth = np.trapz(df_normal_smooth, axis=1)
# _auc_norm_smooth = auc_smooth - min(auc_smooth)
# auc_norm_smooth = _auc_norm_smooth / max(_auc_norm_smooth)

## Smooth Traces

Following steps include subtraction of blank wells, where small random noise can interfere.
Smoothing the traces with Guassian smoothing mitigates the effects of the noise whilst preserving the macro 
features of interest (the P450 peaks)

## Figure: Before and After Smoothing

Or different $\sigma$ paramaters for comparison - need to justify a choice for $\sigma$

## AUC

In [8]:
auc = np.trapz(df_normal, axis=1)
_auc_norm = auc - min(auc)
auc_norm = _auc_norm / max(_auc_norm)


auc_smooth = np.trapz(df_normal_smooth, axis=1)
_auc_norm_smooth = auc_smooth - min(auc_smooth)
auc_norm_smooth = _auc_norm_smooth / max(_auc_norm_smooth)


## Baseline Subtraction

- Each individual trace and corresponding control
- Ideal Baseline - presumed plate inherent absorbance

In [9]:
baseline = df_normal.iloc[auc.argmin(), :]
sigma = 5
baseline_smooth = pd.Series(ndimage.gaussian_filter1d(baseline, sigma=sigma), index=baseline.index)
baseline_smooth -= baseline_smooth[800]

baseline = df_normal_smooth.iloc[auc_norm_smooth.argmin(), :]

df_normal_smooth = df_normal_smooth - baseline_smooth

## Scattering Curve Fitting

In [10]:
# all_rsq = []
# all_k = []
# 
# for i in tqdm(range(len(df_normal_smooth))):
#     row = df_normal_smooth.iloc[i, :]
#     k, cov = optimize.curve_fit(scatter,
#                                 xdata=row.index,
#                                 ydata=row
#                                )
#     y_pred = pd.Series(scatter(row.index, k), index=row.index)
#     rsq = r_squared(row, y_pred)
#     all_rsq.append(rsq)
#     all_k.append(k)
#     
# ks = np.concatenate(all_k).reshape(-1, 1)
# rsqs = np.array(all_rsq)
# 
# mm_scaler = MinMaxScaler()
# mm_scaler.fit(ks)
# power_scaler = PowerTransformer()
# power_scaler.fit(ks)
# 
# def color_k(k):
#     k_norm = power_scaler.transform(np.array(k).reshape(1,1)).flatten()
#     return plt.cm.inferno(k_norm)
#     
# metrics_df = pd.DataFrame([ks.flatten(), rsqs.flatten()], index=['K', 'Rsq']).T
# metrics_df_kslice = metrics_df.loc[metrics_df['Rsq'] > 0, :]
# 
# df_normal_smooth_kslice = df_normal_smooth.iloc[metrics_df_kslice.index, :]

## Exclusion Threshold

In [11]:
configs_df.T

0                  1                  2     \
ligand                              NaN  Arachadionic Acid  Arachadionic Acid   
km                              1.53895         285.406633         337.546311   
vmax                            0.01251           0.079279           0.083692   
rsq                            0.021696          -2.393192          -2.341006   
column_num                          1.0                2.0                3.0   
protein_concentration              4.01               4.01               4.01   
a420_max                          0.093               0.08              0.083   
auc_mean                       8.606125            10.5975           8.540825   
auc_cv                         0.575958           0.443524           0.461805   
std_405                        0.008799           0.012873             0.0055   
dd_soret                       -0.00044          -0.010636          -0.007238   
experiment_number                     5                  5                  5   
plate_type                 Corning 3660       Corning 3660       Corning 3660   
file                   20190619_boi.CSV   20190619_boi.CSV   20190619_boi.CSV   
protein_days_thawed                 NaN                NaN                NaN   
k                                   NaN                NaN                NaN   
% DMSO                              NaN                NaN                NaN   
working_vol                         NaN                NaN                NaN   
controls_ok                         NaN                NaN                NaN   
centrifuge_rpm                      NaN                NaN                NaN   
centrifuge_minutes                  NaN                NaN                NaN   
dispense_ligands                    NaN                NaN                NaN   
well_volume                         NaN                NaN                NaN   
dispense_bulk                       NaN                NaN                NaN   
picklist                            NaN                NaN                NaN   
hours_before_reading                NaN                NaN                NaN   
concentrations                      NaN                NaN                NaN   
percent_dmso                        NaN                NaN                NaN   
Triton  / %                         NaN                NaN                NaN   
DMSO /%                             NaN                NaN                NaN   
Arginine / mM                       NaN                NaN                NaN   
Glucose / mM                        NaN                NaN                NaN   
KPi / mM                            NaN                NaN                NaN   
KCl / mM                            NaN                NaN                NaN   
NaCl / mM                           NaN                NaN                NaN   
NH4 / mM                            NaN                NaN                NaN   
BSA /mg/ml                          NaN                NaN                NaN   
Kpi /mM                             NaN                NaN                NaN   
Kcl/mM                              NaN                NaN                NaN   
shaking_seconds                     NaN                NaN                NaN   
BSA                                 NaN                NaN                NaN   
protein_conc                        NaN                NaN                NaN   
protein_vol                         NaN                NaN                NaN   
ligand_dispensing                   NaN                NaN                NaN   
bulk_dispensing                     NaN                NaN                NaN   
protein_volume                      NaN                NaN                NaN   
experiments                         NaN                NaN                NaN   
column                              NaN                NaN                NaN   

                                    3                  4     \
li